In [8]:
import pandas as pd
pd.options.mode.chained_assignment = None

import plotly.express as px

import lmfit

In [9]:
covid_stats=pd.read_csv("../../covid-statistics-by-us-states-daily-updates.csv")

In [10]:
covid_stats=covid_stats[covid_stats.state=='NY']
covid_stats=covid_stats.drop(['Unnamed: 0', 'posneg', 'pending', 'hash','commercialscore', 'negativeregularscore',
                             'negativescore', 'positivescore', 'grade', 'score','state','dataqualitygrade',
                              'lastupdateet', 'datemodified','checktimeet', 'datechecked', 'fips', 'hash',
                              'commercialscore','negativeregularscore', 'negativescore', 'positivescore',
                              'score', 'grade','onventilatorcumulative','deathconfirmed','deathprobable',
                             'inicucumulative','negativetestsviral','positivetestsviral', 'totaltestsviral',
                              'positivecasesviral'], 
                             axis=1).sort_values('date')

In [11]:
covid_stats = covid_stats.fillna(0)
covid_stats

,date,positive,negative,hospitalizedcurrently,hospitalizedcumulative,inicucurrently,onventilatorcurrently,recovered,death,hospitalized,positiveincrease,negativeincrease,total,totaltestresults,totaltestresultsincrease,deathincrease,hospitalizedincrease
8138,2020-03-04,6.0,48.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,78,54,0,0,0
8120,2020-03-05,22.0,76.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,16,28,122,98,44,0,0
8093,2020-03-06,33.0,92.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,11,16,361,125,27,0,0
8053,2020-03-07,76.0,92.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,43,0,404,168,43,0,0
8003,2020-03-08,105.0,92.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,29,0,197,197,29,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
261,2020-07-25,411200.0,5105111.0,646.0,89995.0,149.0,94.0,72632.0,25103.0,89995.0,750,70716,5516311,5516311,71466,13,0
205,2020-07-26,411736.0,5158143.0,637.0,89995.0,155.0,90.0,72716.0,25106.0,89995.0,536,53032,5569879,5569879,53568,3,0
149,2020-07-27,412344.0,5214805.0,642.0,89995.0,149.0,84.0,72766.0,25117.0,89995.0,608,56662,5627149,5627149,57270,11,0
93,2020-07-28,412878.0,5271668.0,648.0,89995.0,152.0,81.0,72813.0,25126.0,89995.0,534,56863,5684546,5684546,57397,9,0


In [12]:
active = covid_stats['positive'].values - covid_stats['recovered'].values - covid_stats['death'].values
deaths = covid_stats['death'].values
deaths_increase covid_stats['deathincrease'].values
recovered = covid_stats['recovered'].values
hospital_occupancy = covid_stats['hospitalizedcurrently'].values
icu_occupancy = covid_stats['inicucurrently'].values
ventilator_occupancy = covid_stats['onventilatorcurrently'].values
positive = covid_stats['positive'].values

In [13]:
fig = px.line(x=covid_stats['date'],
              y=positive,
              title='positive cases')
fig.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# The SIR model differential equations.
def deriv(y, t, beta, gamma, sigma, N, p_I_to_C, p_C_to_D,ho,io,vo,r_I_to_C,r_C_to_D):
    S, E, I, C, R, D = y

    dSdt = -beta(t)*I*S/N
    dEdt = beta(t)*I*S/N - sigma*1*E
    dIdt = (sigma*1*E) - (r_I_to_C(t)*p_I_to_C*ho[t]) - (gamma*(1-p_I_to_C)*(I-ho[t]))
    dCdt = (r_I_to_C(t)*p_I_to_C*ho[t]) - (r_C_to_D(t)*p_C_to_D*vo[t]) - r_C_to_R(t)*(1-p_C_to_D)*vo[t] - (1*1*(io-vo))
    dRdt = (gamma*(1-p_I_to_C)*I-ho[t]) + r_C_to_R(t)*(1-p_C_to_D)*vo[t]
    dDdt = (r_C_to_D(t)*p_C_to_D*vo[t]) + (1*1*(io-vo))
    return dSdt, dEdt, dIdt, dCdt, dRdt, dDdt

In [ ]:
def Model(beta, gamma,active,deaths,deaths_increase,recovered,hospital_occupancy,icu_occupancy,ventilator_occupancy,positive):
    
    def r_I_to_C(t):
        return(ventilator_occupancy[t]/active[t])
    
    def r_C_to_D(t):
        return(deaths_increase[t]/ventilator_occupancy[t])
    
    def r_C_to_R(t):
        return(recovered[t]/np.sum(ventilator_occupancy[:t+1]))
    
    # Total population, N.
    N = 19453556
    # Initial number of infected and recovered individuals, I0 and R0.
    I0, R0 = 1, 0
    # Everyone else, S0, is susceptible to infection initially.
    S0 = N - I0 - R0
    # Contact rate, beta, and mean recovery rate, gamma, (in 1/days).
    t = np.linspace(0, 148, 148,dtype=int)
    y0 = S0, I0, R0
    ret = odeint(deriv, y0, t, N, args = (beta,gamma))
    S, I, R = ret.T
    return t, S, I, R

In [ ]:
def fitter(x, beta, gamma):
    ret = Model(beta,gamma)
    return ret[3][x]

In [ ]:
mod = lmfit.Model(fitter)
mod.set_param_hint("beta", value = 0.7, vary = True)
mod.set_param_hint("gamma", value = 0.2 , vary = True)
params = mod.make_params()
t = np.linspace(0, 148, 148,dtype=int)
result = mod.fit(data, params, method="least_squares", x=t)
# result